In [ ]:
#Question 3. 
''' Write a program to read all data from CSV file with column headings.
        +--------------+-----------+-------+
        |Employee Name |Department |Salary |
        +--------------+-----------+-------+
        |         James|      Sales|   3000|
        |       Michael|      Sales|   4600|
        |        Robert|      Sales|   4100|
        |         Maria|    Finance|   3000|
        |         James|      Sales|   3000|
        |         Scott|    Finance|   3300|
        |           Jen|    Finance|   3900|
        |          Jeff|  Marketing|   3000|
        |         Kumar|  Marketing|   2000|
        |          Saif|      Sales|   4100|
        +--------------+-----------+-------+
b) Add new column “Bonus” by calculating 5% of Sales Department, 3% of Marketing Department, 4% of Finance
Department.
c) Removing the duplicates entries by 2 ways in Pyspark.
d) Display table by grouping up Department Names and count the total no. of employees from each department.
e) Display table with Ascending order by “employee_name”, then descending by “salary”
f) Crosstab display with align “department” as row, “employee_name” as column and “salary” data to filling up
the table.

g) Write a program to add a new column “salary_rank” by keeping rank from Highest to lowest.
a. [let say, add 1 in “salary_rank” column for highest salary employee, then add 2 for 2nd highest salary
employee]
h) Display records who “employee_name” starts with alphabet “J” by use of SQL Query in Pyspark.
i) Join the below table with the above table by “employee_name” column (only common records]:
+-------------+---------+
Employee Name |   Grade |
+-------------+---------+
|        James|       A1|
|      Michael|       C2|
|       Robert|       C1|
|         Saif|       B2|
|        Rocky|       C1|
|          Sam|       C1|
+-------------+---------+
'''

In [13]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, when, rank, desc, first

In [2]:
spark = SparkSession.builder.appName("Spark Assessment3").getOrCreate()

24/04/22 15:17:33 WARN Utils: Your hostname, Shivanis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.106 instead (on interface en0)
24/04/22 15:17:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/22 15:17:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def read_csv_with_header(file_path):
    try:
        df = spark.read.csv(file_path, header=True)
        return df
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

In [4]:
employee_data_df = read_csv_with_header("sparkassessment3.csv")
employee_data_df.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [5]:
employee_data_df = employee_data_df.withColumn(
    "bonus",
    when(col('department') == "Sales", col('salary') * 0.05)
    .when(col('department') == "Finance", col('salary') * 0.04)
    .when(col('department') == "Marketing", col('salary') * 0.03)
    .otherwise(lit(0))
)
employee_data_df.show()

+-------------+----------+------+-----+
|employee_name|department|salary|bonus|
+-------------+----------+------+-----+
|        James|     Sales|  3000|150.0|
|      Michael|     Sales|  4600|230.0|
|       Robert|     Sales|  4100|205.0|
|        Maria|   Finance|  3000|120.0|
|        James|     Sales|  3000|150.0|
|        Scott|   Finance|  3300|132.0|
|          Jen|   Finance|  3900|156.0|
|         Jeff| Marketing|  3000| 90.0|
|        Kumar| Marketing|  2000| 60.0|
|         Saif|     Sales|  4100|205.0|
+-------------+----------+------+-----+



In [6]:
employee_data_df = employee_data_df.dropDuplicates(["employee_name"])
employee_data_df.show()
#employee_data_df = employee_data_df.distinct()  second method 

+-------------+----------+------+-----+
|employee_name|department|salary|bonus|
+-------------+----------+------+-----+
|        James|     Sales|  3000|150.0|
|         Jeff| Marketing|  3000| 90.0|
|          Jen|   Finance|  3900|156.0|
|        Kumar| Marketing|  2000| 60.0|
|        Maria|   Finance|  3000|120.0|
|      Michael|     Sales|  4600|230.0|
|       Robert|     Sales|  4100|205.0|
|         Saif|     Sales|  4100|205.0|
|        Scott|   Finance|  3300|132.0|
+-------------+----------+------+-----+



In [9]:
department_counts = employee_data_df.groupBy("department").count()
department_counts.show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|    4|
|   Finance|    3|
| Marketing|    2|
+----------+-----+



In [10]:
sorted_employee_data_df = employee_data_df.orderBy(col("employee_name").asc(), col("salary").desc())
sorted_employee_data_df.show()

+-------------+----------+------+-----+
|employee_name|department|salary|bonus|
+-------------+----------+------+-----+
|        James|     Sales|  3000|150.0|
|         Jeff| Marketing|  3000| 90.0|
|          Jen|   Finance|  3900|156.0|
|        Kumar| Marketing|  2000| 60.0|
|        Maria|   Finance|  3000|120.0|
|      Michael|     Sales|  4600|230.0|
|       Robert|     Sales|  4100|205.0|
|         Saif|     Sales|  4100|205.0|
|        Scott|   Finance|  3300|132.0|
+-------------+----------+------+-----+



In [11]:
salary_by_department = employee_data_df.groupBy("department").pivot("employee_name").agg(first("salary"))
salary_by_department.show()

+----------+-----+----+----+-----+-----+-------+------+----+-----+
|department|James|Jeff| Jen|Kumar|Maria|Michael|Robert|Saif|Scott|
+----------+-----+----+----+-----+-----+-------+------+----+-----+
|   Finance| null|null|3900| null| 3000|   null|  null|null| 3300|
| Marketing| null|3000|null| 2000| null|   null|  null|null| null|
|     Sales| 3000|null|null| null| null|   4600|  4100|4100| null|
+----------+-----+----+----+-----+-----+-------+------+----+-----+



In [14]:
window_spec = Window.orderBy(col("salary").desc())
employee_data_df = employee_data_df.withColumn("salary_rank", rank().over(window_spec))

In [15]:
employee_data_df.show()

24/04/22 15:24:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:24:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:24:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:24:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:24:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:24:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------+----------+------+-----+-----------+
|employee_name|department|salary|bonus|salary_rank|
+-------------+----------+------+-----+-----------+
|      Michael|     Sales|  4600|230.0|          1|
|       Robert|     Sales|  4100|205.0|          2|
|         Saif|     Sales|  4100|205.0|          2|
|          Jen|   Finance|  3900|156.0|          4|
|        Scott|   Finance|  3300|132.0|          5|
|        James|     Sales|  3000|150.0|          6|
|         Jeff| Marketing|  3000| 90.0|          6|
|        Maria|   Finance|  3000|120.0|          6|
|        Kumar| Marketing|  2000| 60.0|          9|
+-------------+----------+------+-----+-----------+



In [17]:
employee_data_df.createOrReplaceTempView("employee_data")
j_name_employees = spark.sql("SELECT * FROM employee_data WHERE employee_name LIKE 'J%'")
j_name_employees.show()

24/04/22 15:25:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:25:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:25:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:25:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:25:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:25:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 1

+-------------+----------+------+-----+-----------+
|employee_name|department|salary|bonus|salary_rank|
+-------------+----------+------+-----+-----------+
|          Jen|   Finance|  3900|156.0|          4|
|        James|     Sales|  3000|150.0|          6|
|         Jeff| Marketing|  3000| 90.0|          6|
+-------------+----------+------+-----+-----------+



In [21]:
data = [
    ("James", "A1"),
    ("Michael", "C2"),
    ("Robert", "C1"),
    ("Saif", "B2"),
    ("Rocky", "C1"),
    ("Sam", "C1")
]
grade_df = spark.createDataFrame(data, ["Employee_Name", "grade"])

In [22]:
grade_df.show()

+-------------+-----+
|Employee_Name|grade|
+-------------+-----+
|        James|   A1|
|      Michael|   C2|
|       Robert|   C1|
|         Saif|   B2|
|        Rocky|   C1|
|          Sam|   C1|
+-------------+-----+



In [24]:
joined_df = employee_data_df.join(
    grade_df,
    on=employee_data_df["employee_name"] == grade_df["Employee_Name"],
    how="inner").drop(grade_df["Employee_Name"])

joined_df.show()

# # Select desired columns
# joined_df = joined_df.select(
#     employee_data_df["Employee Name"],
#     employee_data_df["Department"],
#     employee_data_df["Salary"],
#     employee_data_df["Bonus"],
#     employee_data_df["salary_rank"],
#     grade_data_df["Grade"]
# )


24/04/22 15:32:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:32:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:32:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:32:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:32:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/22 15:32:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------+----------+------+-----+-----------+-----+
|employee_name|department|salary|bonus|salary_rank|grade|
+-------------+----------+------+-----+-----------+-----+
|        James|     Sales|  3000|150.0|          6|   A1|
|      Michael|     Sales|  4600|230.0|          1|   C2|
|       Robert|     Sales|  4100|205.0|          2|   C1|
|         Saif|     Sales|  4100|205.0|          2|   B2|
+-------------+----------+------+-----+-----------+-----+

